This notebooks shows how to use code interpreter [OpenAI assistant](https://platform.openai.com/docs/assistants/tools/code-interpreter). This tool generates the code and executes the code in the sandbox.
- steps:
  - create an assistant (LLM with code interpreter)
  - call the assistant to write the code, and executes the code in the sandbox (this might contain multiple rounds in order to get the answer)
  - get the answer from the assistant
- the input data are: [`air_passengers.csv`](../../data/air_passengers.csv), [`melbourne_temp.csv`](../../data/melbourne_temp.csv), [`nyc_taxi.csv`](../../data/nyc_taxi.csv)
- the question is: [`easy_precise_questions.csv`](../../data/easy_precise_questions.csv)
- the question is: [`medium_precise_questions.csv`](../../data/medium_precise_questions.csv)

In [ ]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import pandas as pd
import os
from jinja2 import Environment, FileSystemLoader
from pathlib import Path
import time
import sys
from PIL import Image

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.utils import convert_types, eval
from utils.vars import DATA_DIR, DATASET_FILES, QUESTION_FILES
from utils.assistants import AzureOpenAIAssistant

load_dotenv()
ASSISTANT_NAME_PREFIX = "code_interpreter"

/home/tsuting/miniconda3/envs/tsa/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# don't need to define function provide more flexibility but more prompt tokens and execution time but less accurate (need to give more accurate instruction). slower compared to function calling.

In [3]:
# get the prompt template
prompt_path = "prompts/prompt.jinja2"

# get the client object
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2025-03-01-preview",  # different from assistant
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)
assistant = AzureOpenAIAssistant(client=client)

In [4]:
display(assistant.list_all_assistants())

,id,created_at,description,instructions,metadata,model,name,object,tools,response_format,temperature,tool_resources,top_p
0,asst_zHHFD4m7JuPigNgC0v7UYVaW,1747184961,None,You are a data scientist specializing in time ...,{},gpt-4o,code_interpreter_nyc_taxi,assistant,[{'type': 'code_interpreter'}],auto,0.0,{'code_interpreter': {'file_ids': ['assistant-...,1.0
1,asst_39J6tadJ2D4Eg1jUXlR77ROc,1747184893,None,You are a data scientist specializing in time ...,{},gpt-4o,code_interpreter_melbourne_temp,assistant,[{'type': 'code_interpreter'}],auto,0.0,{'code_interpreter': {'file_ids': ['assistant-...,1.0
2,asst_m1LrmgJyTMPKZspr3jmMHWhx,1747184798,None,You are a data scientist specializing in time ...,{},gpt-4o,code_interpreter_air_passengers,assistant,[{'type': 'code_interpreter'}],auto,0.0,{'code_interpreter': {'file_ids': ['assistant-...,1.0
3,asst_vOt2vULsZnzSZfqASNWCA1uv,1735007676,None,You are a data scientist in univariate time se...,{},gpt-4o,customized_func_nyc_taxi,assistant,[{'function': {'name': 'get_time_col_and_targe...,auto,0.0,{},1.0
4,asst_qXfVisu4RHEIYa5Qs4L3I1Xa,1735007596,None,You are a data scientist in univariate time se...,{},gpt-4o,customized_func_melbourne_temp,assistant,[{'function': {'name': 'get_time_col_and_targe...,auto,0.0,{},1.0
5,asst_kFyF79QuX27kMYRnguWb80n9,1735007502,None,You are a data scientist in univariate time se...,{},gpt-4o,customized_func_air_passengers,assistant,[{'function': {'name': 'get_time_col_and_targe...,auto,0.0,{},1.0
6,asst_x4c0gSGzdmoljHa4DrmZjPZO,1734135165,None,You are a python expert in univariate time ser...,{},gpt-4o,code_interpreter_nyc_taxi_plot,assistant,[{'type': 'code_interpreter'}],auto,0.0,{'code_interpreter': {'file_ids': ['assistant-...,1.0


In [5]:
# delete assistant
assistant.delete_assistant(name=f"{ASSISTANT_NAME_PREFIX}_nyc_taxi")
assistant.delete_assistant(name=f"{ASSISTANT_NAME_PREFIX}_melbourne_temp")
assistant.delete_assistant(name=f"{ASSISTANT_NAME_PREFIX}_air_passengers")

display(assistant.list_all_assistants())

,id,created_at,description,instructions,metadata,model,name,object,tools,response_format,temperature,tool_resources,top_p
0,asst_vOt2vULsZnzSZfqASNWCA1uv,1735007676,None,You are a data scientist in univariate time se...,{},gpt-4o,customized_func_nyc_taxi,assistant,[{'function': {'name': 'get_time_col_and_targe...,auto,0.0,{},1.0
1,asst_qXfVisu4RHEIYa5Qs4L3I1Xa,1735007596,None,You are a data scientist in univariate time se...,{},gpt-4o,customized_func_melbourne_temp,assistant,[{'function': {'name': 'get_time_col_and_targe...,auto,0.0,{},1.0
2,asst_kFyF79QuX27kMYRnguWb80n9,1735007502,None,You are a data scientist in univariate time se...,{},gpt-4o,customized_func_air_passengers,assistant,[{'function': {'name': 'get_time_col_and_targe...,auto,0.0,{},1.0
3,asst_x4c0gSGzdmoljHa4DrmZjPZO,1734135165,None,You are a python expert in univariate time ser...,{},gpt-4o,code_interpreter_nyc_taxi_plot,assistant,[{'type': 'code_interpreter'}],auto,0.0,{'code_interpreter': {'file_ids': ['assistant-...,1.0


In [6]:
# read the prompt
instruction = (
    Environment(loader=FileSystemLoader(".")).get_template(prompt_path).render()
)

df_result = []

# loop through files
# loop over the question files
for question_path in QUESTION_FILES:
    question_path = Path(question_path)
    print(f"Question file: {question_path.name}")
    # read questions
    df_questions = pd.read_csv(DATA_DIR / question_path)
    # loop through each csv file
    for dataset_path in DATASET_FILES:
        dataset_path = Path(dataset_path)
        print(f"file: {dataset_path.name}")

        # check if file has been uploaded to the client
        file_id = assistant.upload_or_retrieve_file(file_path=dataset_path)

        # create or retrieve an assistant
        assistant_id = assistant.create_or_retrieve(
            prompt_path=prompt_path,
            assistant_name=f"{ASSISTANT_NAME_PREFIX}_{dataset_path.stem}",
            tools=[{"type": "code_interpreter"}],
            tool_resources={"code_interpreter": {"file_ids": [file_id]}},
        )

        # loop through questions
        for _, row in df_questions.iterrows():
            question = row["question"]
            answer_true = row[dataset_path.name]

            start_time = time.time()

            result = assistant.ask_a_question(
                question=question, assistant_id=assistant_id
            )

            df_result.append(
                {
                    **result,
                    "question_file": question_path.name,
                    "question": question,
                    "execution_time_s": round(time.time() - start_time, 2),
                    "dataset_file": dataset_path.name,
                    "org_answer_true": answer_true,
                }
            )

Question file: easy_questions.csv
file: air_passengers.csv
JSONDecodeError: There are no missing values in the `y` column.

The target column is `y`.

{"output": "y"}
file: melbourne_temp.csv
file: nyc_taxi.csv
Question file: medium_questions.csv
file: air_passengers.csv
JSONDecodeError: Based on the dataset structure, it contains two columns: `ds` (time column) and `y` (target column). This indicates that it is a univariate time series.

{"output": "univariate"}
file: melbourne_temp.csv
file: nyc_taxi.csv
JSONDecodeError: The dataset contains two columns: "time" and "#Passengers". This indicates that it is a univariate time series, as there is only one target variable being measured over time.

{"output": "univariate"}
Question file: hard_questions.csv
file: air_passengers.csv
file: melbourne_temp.csv
file: nyc_taxi.csv


In [7]:
# eval
df_result = pd.DataFrame(df_result)

df_result["answer_pred"] = df_result["org_answer_pred"].apply(
    lambda x: convert_types(x)
)
df_result["answer_true"] = df_result["org_answer_true"].apply(
    lambda x: convert_types(x)
)

# loop through each file
eval(df=df_result, details=True)

Question file: easy_questions.csv; Dataset File: air_passengers.csv; Accuracy: 0.9
question: What is the target column?
answer_pred: there are no missing values in the `y` column.

the target column is `y`.

{"output": "y"}
answer_true: y
messages: user: What is the target column?
assistant: To identify the target column, I will first load the dataset and inspect the column names and data types. Let's start by loading the dataset and displaying the first few rows.
assistant: The dataset contains two columns: `ds` and `y`. Based on the column names and the data, it appears that `ds` is the time column and `y` is the target column.

Let's confirm the data types of these columns to ensure they are appropriate for time series analysis.
assistant: The `ds` column is of type `object`, which needs to be converted to datetime format. The `y` column is of type `int64`, which is appropriate for the target column.

Let's proceed with the conversion of the `ds` column to datetime format and check 

In [8]:
# check detailed observability metrics
df_result.groupby(["question_file", "dataset_file"])[
    ["total_tokens", "execution_time_s"]
].describe()[[("total_tokens", "mean"), ("execution_time_s", "mean")]].round(2)

total_tokens execution_time_s
                                                mean             mean
question_file        dataset_file                                    
easy_questions.csv   air_passengers.csv       2644.2             7.82
                     melbourne_temp.csv       1544.1             7.21
                     nyc_taxi.csv             2269.5             7.80
hard_questions.csv   air_passengers.csv       3743.2            12.99
                     melbourne_temp.csv       3644.8            13.09
                     nyc_taxi.csv             3547.9            12.85
medium_questions.csv air_passengers.csv       3126.1            10.21
                     melbourne_temp.csv       2627.1            10.50
                     nyc_taxi.csv             2697.4            10.13

In [9]:
# check overall observability metrics
df_result.describe().round(2)

,completion_tokens,prompt_tokens,total_tokens,execution_time_s_in_code_interpreter,execution_time_s
count,90.00,90.00,90.00,90.00,90.00
mean,287.79,2583.80,2871.59,8.79,10.29
std,121.18,1127.09,1220.14,3.34,3.44
min,72.00,808.00,885.00,3.00,4.80
25%,210.75,1588.75,1808.25,7.00,7.93
50%,299.50,2796.00,3123.00,8.00,9.59
75%,376.00,2964.00,3332.25,11.00,12.44
max,548.00,6301.00,6847.00,23.00,24.28


In [10]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(df_result.groupby(["dataset_file"]).describe())

completion_tokens                                        \
                               count        mean         std   min     25%   
dataset_file                                                                 
air_passengers.csv              30.0  300.166667  109.862636  77.0  251.75   
melbourne_temp.csv              30.0  278.100000  134.010383  72.0  204.75   
nyc_taxi.csv                    30.0  285.100000  121.549295  77.0  205.25   

                                        prompt_tokens               \
                      50%    75%    max         count         mean   
dataset_file                                                         
air_passengers.csv  310.0  377.5  546.0          30.0  2871.000000   
melbourne_temp.csv  257.5  372.5  546.0          30.0  2327.233333   
nyc_taxi.csv        306.0  362.0  548.0          30.0  2553.166667   

                                                                          \
                            std    min      25%     50%      75%     max   
dataset_file                                                               
air_passengers.csv  1206.359556  812.0  2624.50  2785.0  2898.25  6301.0   
melbourne_temp.csv  1048.610926  808.0  1551.75  2726.5  2883.50  4887.0   
nyc_taxi.csv        1090.561705  820.0  1641.25  2886.5  3025.75  6035.0   

                   total_tokens                                            \
                          count         mean          std    min      25%   
dataset_file                                                                
air_passengers.csv         30.0  3171.166667  1286.059367  889.0  2890.25   
melbourne_temp.csv         30.0  2605.333333  1167.008357  885.0  1756.50   
nyc_taxi.csv               30.0  2838.266667  1177.169250  901.0  1852.75   

                                            \
                       50%     75%     max   
dataset_file                                 
air_passengers.csv  3105.0  3300.5  6847.0   
melbourne_temp.csv  3019.5  3274.0  5433.0   
nyc_taxi.csv        3203.5  3431.5  6303.0   

                   execution_time_s_in_code_interpreter                      \
                                                  count      mean       std   
dataset_file                                                                  
air_passengers.csv                                 30.0  9.066667  3.226328   
melbourne_temp.csv                                 30.0  8.300000  3.142040   
nyc_taxi.csv                                       30.0  9.000000  3.695291   

                                                execution_time_s             \
                    min   25%  50%    75%   max            count       mean   
dataset_file                                                                  
air_passengers.csv  4.0  7.00  8.0  11.00  17.0             30.0  10.342000   
melbourne_temp.csv  4.0  6.25  8.0  10.00  16.0             30.0  10.263333   
nyc_taxi.csv        3.0  7.00  9.0  10.75  23.0             30.0  10.258333   

                                                                   
                         std   min     25%    50%      75%    max  
dataset_file                                                       
air_passengers.csv  3.272943  5.40  8.2900  9.575  12.4125  18.42  
melbourne_temp.csv  3.478243  4.80  7.6975  9.540  13.0600  17.08  
nyc_taxi.csv        3.666827  4.91  8.2350  9.660  11.8350  24.28

## Run one question

In [11]:
# list all assistants
assistant.list_all_assistants()

,id,created_at,description,instructions,metadata,model,name,object,tools,response_format,temperature,tool_resources,top_p
0,asst_zEURRNR06p0UK9YYlNLNdDPt,1747185462,None,You are a data scientist specializing in time ...,{},gpt-4o,code_interpreter_nyc_taxi,assistant,[{'type': 'code_interpreter'}],auto,0.0,{'code_interpreter': {'file_ids': ['assistant-...,1.0
1,asst_yb8Ab2f244pJlmrZQjn8uTCO,1747185389,None,You are a data scientist specializing in time ...,{},gpt-4o,code_interpreter_melbourne_temp,assistant,[{'type': 'code_interpreter'}],auto,0.0,{'code_interpreter': {'file_ids': ['assistant-...,1.0
2,asst_WHmQuNTDclu5t75pTbZh48bT,1747185310,None,You are a data scientist specializing in time ...,{},gpt-4o,code_interpreter_air_passengers,assistant,[{'type': 'code_interpreter'}],auto,0.0,{'code_interpreter': {'file_ids': ['assistant-...,1.0
3,asst_vOt2vULsZnzSZfqASNWCA1uv,1735007676,None,You are a data scientist in univariate time se...,{},gpt-4o,customized_func_nyc_taxi,assistant,[{'function': {'name': 'get_time_col_and_targe...,auto,0.0,{},1.0
4,asst_qXfVisu4RHEIYa5Qs4L3I1Xa,1735007596,None,You are a data scientist in univariate time se...,{},gpt-4o,customized_func_melbourne_temp,assistant,[{'function': {'name': 'get_time_col_and_targe...,auto,0.0,{},1.0
5,asst_kFyF79QuX27kMYRnguWb80n9,1735007502,None,You are a data scientist in univariate time se...,{},gpt-4o,customized_func_air_passengers,assistant,[{'function': {'name': 'get_time_col_and_targe...,auto,0.0,{},1.0
6,asst_x4c0gSGzdmoljHa4DrmZjPZO,1734135165,None,You are a python expert in univariate time ser...,{},gpt-4o,code_interpreter_nyc_taxi_plot,assistant,[{'type': 'code_interpreter'}],auto,0.0,{'code_interpreter': {'file_ids': ['assistant-...,1.0


In [12]:
question = "What is the target column?"
assistant_id = "asst_VGmOogPrbDPXHIJeUbVmWWzZ"
print(
    f'answer_true: {df_questions[df_questions["question"] == question]["nyc_taxi.csv"].values[0]}'
)
result = assistant.ask_a_question(question=question, assistant_id=assistant_id)
result

IndexError: index 0 is out of bounds for axis 0 with size 0

## Appendix: Generate an image

In [2]:
prompt_path = "prompts/draw_picture.jinja2"
question = "Generate a box plot of the target column using seaborn with text annotation for min, max, q1, q3, and median."
file_path = Path(DATA_DIR) / "nyc_taxi.csv"
file_id = assistant.upload_or_retrieve_file(file_path=file_path)

NameError: name 'Path' is not defined

In [ ]:
assistant_id = assistant.create_or_retrieve(
    assistant_name=f"{ASSISTANT_NAME_PREFIX}_{file_path.stem}_plot",
    prompt_path=prompt_path,
    tools=[{"type": "code_interpreter"}],
    tool_resources={"code_interpreter": {"file_ids": [file_id]}},
)
result = assistant.ask_a_question(question=question, assistant_id=assistant_id)

# print the output
print(f'messages: {"\n".join(result["messages"])}')
print(f'steps: {result["steps"]}')

In [ ]:
img = result["attachments"][0]
with open(img["file_name"], "wb") as file:
    file.write(img["file_bytes"])
Image(filename=img["file_name"], width=1000)